In [1]:
import modules
import answerer
from dataset import MyDataset

In [2]:
from types import SimpleNamespace
d = {
    "data_path": "/shared/shang/datasets/nextqa/videos/",
    "query_file": "/shared/shang/datasets/nextqa/metadata/splits/val/val_queries_t.csv",
    "start_sample": 0,
    "max_samples": 2000,
    "gpu1": 4,
    "gpu2": 5,
}

args = SimpleNamespace(**d)
args

namespace(data_path='/shared/shang/datasets/nextqa/videos/',
          query_file='/shared/shang/datasets/nextqa/metadata/splits/val/val_queries_t.csv',
          start_sample=0,
          max_samples=2000,
          gpu1=4,
          gpu2=5)

In [3]:
import pandas as pd
val_temporal = pd.read_csv('/shared/shang/datasets/nextqa/metadata/splits/val/val_queries_t.csv')
val_temporal[val_temporal['query_type'] == 'TN'][:10]

,index,sample_id,possible_answers,query_type,query,answer,video_name
0,0,3,"['drink again', 'shake its body', 'smells the ...",TN,what does the white dog do after going to the ...,2,2834146886.mp4
1,0,7,"['put hand in mouth', 'continue skating', 'jum...",TN,what does the female skater do after the male ...,1,3441428429.mp4
2,0,8,"['grab her', 'feed horse with grass', 'run tow...",TN,what does the girl in white do after bending d...,1,6356067859.mp4
4,0,13,"['look at him', 'touch their chests', 'kick hi...",TN,what does the man do after the lady appear to ...,3,5296635780.mp4
6,0,20,"['open her eyes', 'move toward the slides', 't...",TN,what does the baby do after letting go of the ...,1,6136926089.mp4
7,0,23,"['sing', 'walk forward and observe', 'hit cans...",TN,what does the man in white do after moving for...,2,6018490041.mp4
9,0,27,"['smile and wants to pet it', 'backed away', '...",TN,what does the dog do after the baby touches an...,1,7416295940.mp4
10,0,29,"['hug the girl', 'smile', 'stand up', 'continu...",TN,what did the girl do after she finished reciti...,2,3943634344.mp4
12,0,39,"['mount the black dog', 'look around', 'looks ...",TN,what does the brown dog do after stepping over...,1,4094488636.mp4
13,0,45,"['move his legs', 'suck his thumb', 'raised hi...",TN,what did the baby do after he approached near ...,2,3429509208.mp4


In [4]:
dataset = MyDataset(data_path=args.data_path,
                    query_file=args.query_file,
                    start_sample=args.start_sample,
                    max_samples=args.max_samples)

In [5]:
siglip = modules.SiglipModel(gpu_number=args.gpu1, siglip_model_type="ViT-B-16-SigLIP")
llava = modules.LLAVA(gpu_number=args.gpu2)
llm = modules.GPTModel()
ans = answerer.Answerer(llava, llava, siglip, llm)

batch_correct = 0
total_correct = 0

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
item0 = dataset[0]
video0 = dataset.construct_video(item0)
print(item0['video_name'])
print(item0['query'])
print(item0['possible_answers'])
print('correct answer', item0['answer'])

pred0 = ans.forward(video0)
pred0

2834146886.mp4
what does the white dog do after going to the cushion?
['drink again', 'shake its body', 'smells the black dog', 'wagging tail', 'touch lady in blue stripes']
correct answer 2


SyntaxError: invalid syntax (<unknown>, line 1)

In [7]:
# correct
item1 = dataset[1]
video1 = dataset.construct_video(item1)
print(item1['video_name'])
print(item1['query'])
print(item1['possible_answers'])
print('correct answer:', item1['answer'])

pred1 = ans.forward(video1)
pred1

3441428429.mp4
what does the female skater do after the male skater puts her back down on the ice?
['put hand in mouth', 'continue skating', 'jump', 'move her arms up and down', 'laugh and run forward']
correct answer: 1


1

In [8]:
# correct
item2 = dataset[2]
video2 = dataset.construct_video(item2)
print(item2['video_name'])
print(item2['query'])
print(item2['possible_answers'])
print('correct answer:', item2['answer'])

pred2 = ans.forward(video2)
pred2

6356067859.mp4
what does the girl in white do after bending down in the middle?
['grab her', 'feed horse with grass', 'run towards the camera', 'umbrella', 'put her arms up']
correct answer: 1


1

In [13]:
# incorrect
# predicts None insetad of 3
item3 = dataset[4]
video3 = dataset.construct_video(item3)
print(item3['video_name'])
print(item3['query'])
print(item3['possible_answers'])
print('correct answer:', item3['answer'])

pred3 = ans.forward(video3)
pred3

5296635780.mp4
what does the man do after the lady appear to punch him and smiles?
['look at him', 'touch their chests', 'kick him', 'lean forward', 'sit back properly']
correct answer: 3


3

In [14]:
# incorrect
# predicts 2 instead of 1
item4 = dataset[6]
video4 = dataset.construct_video(item4)
print(item4['video_name'])
print(item4['query'])
print(item4['possible_answers'])
print('correct answer:', item4['answer'])

pred4 = ans.forward(video4)
pred4

6136926089.mp4
what does the baby do after letting go of the cart?
['open her eyes', 'move toward the slides', 'touch the toy', 'turn to his back', 'lean forward and put head down']
correct answer: 1


1

In [6]:
# incorrect
# predicts 3 instead of 2
item5 = dataset[7]
video5 = dataset.construct_video(item5)
print(item5['video_name'])
print(item5['query'])
print(item5['possible_answers'])
print('correct answer:', item5['answer'])

pred5 = ans.forward(video5)
pred5

6018490041.mp4
what does the man in white do after moving forward at the start?
['sing', 'walk forward and observe', 'hit cans', 'clicks the picture', 'start dancing']
correct answer: 2
{'Explanation': "This question asks about an action that occurs at the start, so we need to start from the beginning of the video. We are currently on the frame at 0.06666666666666667 seconds in a 27.5 second video, so we are nearly at the start. Since we are close to the start, we do not need to travel to another frame. To understand the action of the man in white after moving forward, we need to ask a simple question about the man's action after moving forward to fulfill the plan.", 'Go-To': 0.06666666666666667, 'Questions': ['What is the man in white doing after moving forward?']}
{'Explanation': "Based on the plan and the question, we need to first identify the frame where the man in white moves forward. Since the video is 27.5 seconds long, we can choose to go to a frame around the middle of the vi

1

In [ ]:
item6 = dataset[9]
video6 = dataset.construct_video(item6)
print(item6['video_name'])
print(item6['query'])
print(item6['possible_answers'])
print('correct answer:', item6['answer'])

pred6 = ans.forward(video6)
pred6

In [ ]:
for i in tqdm(range(len(dataset))):
    item = dataset[i]
    video = dataset.construct_video(item)
    try:
        pred = ans.forward(video)
        with open(args.output_file, "a") as outfile:
            outfile.write(f"{item['index']},{item['video_name']},{item['query_type']},{item['query']},{item['answer']},{item['possible_answers']},{pred}\n")
        if pred == item["answer"]:
            print("correct")
            batch_correct += 1
            total_correct += 1
        if i+1 % args.print_interval == 0:
            print("Batch accuracy: ", batch_correct / args.print_interval)
            batch_correct = 0
    except Exception as e:
        print(e)
        pass